# M10: Empirische Wirtschaftsforschung
# Modul 5: Datenwualitätsprobleme

## Vorbereitung

In [ ]:
# Packages laden
install.packages("install.load")
library(install.load)
install_load("ggplot2")

# Daten laden
winequality=read.csv("winequality.csv")

In [ ]:
# Daten inspizieren ----------
dim(winequality) # Anzahl Zeilen und Spalten
head(winequality)

In [ ]:
summary(winequality)

In [ ]:
table(winequality$color)

In [ ]:
# Erstellen von einer Dummy Variable aus color
winequality$red = as.numeric(winequality$color=="red")
# Löschen der nicht mehr verwendeten color Variable
winequality=winequality[,colnames(winequality)!="color"]

head(winequality)

In [ ]:
# Schätzen der Regression
lm_basic = lm(data=winequality, 
              formula="quality ~ alcohol")

summary(lm_basic)

## Messfehler in der unabhängigen Variable

### Zufälliger Messfehler

In [ ]:
# zufälliger Messfehler auf Quality
set.seed(127127)
e=rnorm(nrow(winequality),0,1)

# Neue Outcome-Variable mit Messfehler
winequality$quality_faulty=winequality$quality+e

In [ ]:
# Schätzen der Regression
lm_faulty = lm(data=winequality, 
               formula="quality_faulty ~ alcohol")

summary(lm_faulty)

In [ ]:
# Vergleich des alcohol Schätzwert
b_alcohol_basic=lm_basic$coefficients[names(lm_basic$coefficients)=="alcohol"]
b_alcohol_basic

b_alcohol_faulty=lm_faulty$coefficients[names(lm_faulty$coefficients)=="alcohol"]
b_alcohol_faulty

### Korrelierter Messfehler

In [ ]:
# Korrelierter Messfehler auf Quality
set.seed(127127)
e_cor=rnorm(nrow(winequality),winequality$alcohol-mean(winequality$alcohol),1)

# Neue Outcome-Variable mit Messfehler
winequality$quality_cor=winequality$quality+e_cor

In [ ]:
# Inspektion der Korrelation
ggplot(winequality) + geom_point(aes(x=alcohol, y=e), col="blue")+theme_classic()
ggplot(winequality) + geom_point(aes(x=alcohol, y=e_cor), col="red")+theme_classic()

In [ ]:
# Schätzen der Regression
lm_cor = lm(data=winequality, 
               formula="quality_cor ~ alcohol")

summary(lm_cor)

In [ ]:
# Vergleich des alcohol Schätzwert
b_alcohol_basic=lm_basic$coefficients[names(lm_basic$coefficients)=="alcohol"]
b_alcohol_basic

b_alcohol_cor=lm_cor$coefficients[names(lm_cor$coefficients)=="alcohol"]
b_alcohol_cor

### Simulation der Auswirkungen von einem zufälligen Messfehler

In [ ]:
set.seed(127127)
b_alcohol_vector=c()

In [ ]:
for(i in 1:100){
  # zufälliger Messfehler
  e=rnorm(nrow(winequality),0,1)
  winequality$quality_sim=winequality$quality+e
  
  # Regression
  lm_loop = lm(data=winequality, 
              formula="quality_sim ~ alcohol")
  
  # Schätzwert
  b_alcohol_vector[i]=lm_loop$coefficients[names(lm_loop$coefficients)=="alcohol"]
}

In [ ]:
# Alle Schätzwerte
head(b_alcohol_vector)
ggplot()+ geom_histogram(aes(b_alcohol_vector))+ theme_classic()

In [ ]:
# Theoretischer Mittelwert
b_1 = lm_basic$coefficients[names(lm_basic$coefficients)=="alcohol"]
cov_x_e = 0
var_x1 = var(winequality$alcohol)

b_1+cov_x_e/var_x1

# Praktischer Mittelwert
mean(b_alcohol_vector)

In [ ]:
# Theoretische Varianz
var_u = var(lm_basic$residuals)
var_e = 1
var_x1 = var(winequality$alcohol)
             
(var_u + var_e) / (nrow(winequality) * var_x1)

# Praktische Varianz
var(b_alcohol_vector)

In [ ]:
# Vergleich der Standardabweichung zum Basic Modell
summary(lm_basic)
sd(b_alcohol_vector)

## Messfehler in der abhängigen Variable

Fügen sie einen zufälligen standard-normalverteilten Messfehler zu der Variable *alcohol* hinzu.

In [ ]:
# Ihr Code

Vergleichen Sie den theoretischen Mittelwert und die Varianz des geschätzten Parameters mit den Werten der Simulation (Analog zur vorherigen Übung ‚Messfehler in der abhängigen Variable‘)

In [ ]:
# Ihr Code

## Instrumentalvariable
Wir schätzen den Effekt von dem Packpreis für Zigaretten auf den Zigarettenverkauf mittels der Mehrwertsteuer als Instrument.

In [ ]:
install_load("ivreg")

## data
data("CigaretteDemand", package = "ivreg")

In [ ]:
# Inspizieren der Daten
summary(CigaretteDemand)
?CigaretteDemand

In [ ]:
## model 
iv_model = ivreg(log(packs) ~ log(rprice) + log(rincome) | salestax + log(rincome),
           data = CigaretteDemand)
summary(iv_model)